# Kaggle Feature Engineering microcurse
- Better features make better models. 
- Discover how to get the most out of your data
- https://www.kaggle.com/learn/feature-engineering

## 1.- What Is Feature Engineering
- Learn the steps and principles of creating better features

1. determine which features are the most importan with *mutual information*
2. invent new features in several real-world problem domains
3. encode hisgh-cardinality categoricals with a *target encoding*
4. create segmentation features with *k-means clustering*
5. decompose a dataset´s variation into features with *principal component analysis*

### The Goal of Feature Engineering
- make your data better suited to the problem at hand

1. improve a model´s predictive performance
2. reduce comupational or data needs
3. improve interpretability of the resutls

### A Guiding Principle of Feature Engineering

- For a feature to be useful, it must have a relatoinship to the target that your model es able to learn.
- 
Linear models, for instance, are only able to learn linear relationships
- 
So, when using a liner model, your goal is to transform the features to make their relationship to the target linea
- .
The key ideas here is that a transformation you apply to a feature becomes in essence part of the model itse
- f.
ex. trying to predict Price of square plots of land from de Lenght, the relationship (b/price and Lenght) will be linear  nly if we transform lentght to area (are squares).

### Example - Concrete Formulations
- Add 'synthetic'features can improve the predictive performance of a model.

In [4]:
import zipfile as zfm
import pandas as pd
pd.__version__

'1.5.3'

In [5]:
#url = 'https://github.com/jmonti-gh/Datasets/blob/\
#c790af2d1885dcd63baea8b5a6f9dc8c1b8a1531/Concrete_Data.xls'
# url = 'https://github.com/jmonti-gh/Datasets/blob/main/test.csv'
# df = pd.read_csv(url)
# df

In [6]:
zipfile = 'files/ConcreteCompressiveStrength.zip'
dataset = 'Concrete_Data.xls'

with zfm.ZipFile(zipfile) as zf:
    df_xls = pd.read_excel(zf.open(dataset))

df = pd.read_csv('files/concrete.csv')

print(df.shape)
display(df_xls.iloc[[0, 9, -9, -1]])
display(df.iloc[[0, 9, -9, -1]])

### read_axcel: install xlrd (for xls); install openpyxl (for xlsx)
# https://stackoverflow.com/questions/48066517/python-pandas-pd-read-excel-giving-importerror-install-xlrd-0-9-0-for-excel

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 1.0.0 for Excel support Use pip or conda to install xlrd.

The Concrete dataset contains a variety of concrete formulations and the resulting product's compressive strength, which is a measure of how much load that kind of concrete can bear. The task for this dataset is to predict a concrete's compressive strength given its formulation.

In [7]:
# libraries necesary to build and evaluate the model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

ModuleNotFoundError: No module named 'sklearn'

You can see in the df the various ingredients going into each variety of concrete. We'll see in a moment how adding some additional synthetic features derived from these can help a model to learn important relationships among them.

We'll first establish a baseline by training the model on the un-augmented dataset. This will help us determine whether our new features are actually useful.

Establishing baselines like this is good practice at the start of the feature engineering process. A baseline score can help you decide whether your new features are worth keeping, or whether you should discard them and possibly try something else.

In [8]:
df.columns

NameError: name 'df' is not defined

In [9]:
target = df_xls.columns[-1]
tdf = df.columns[-1]
X = df.copy()
y = X.pop(tdf)



NameError: name 'df_xls' is not defined

In [10]:
# Train and score baseline model
baseline = RandomForestRegressor(criterion="absolute_error", random_state=0)
baseline_score = cross_val_score(
    baseline, X, y, cv=5, scoring="neg_mean_absolute_error"
)
baseline_score = -1 * baseline_score.mean()

print(f"MAE Baseline Score: {baseline_score:.4}")

NameError: name 'RandomForestRegressor' is not defined

If you ever cook at home, you might know that the ratio of ingredients in a recipe is usually a better predictor of how the recipe turns out than their absolute amounts. We might reason then that ratios of the features above would be a good predictor of CompressiveStrength.

The cell below adds three new ratio features to the dataset.

In [11]:
df.columns

NameError: name 'df' is not defined

In [12]:
X = df.copy()
y = X.pop("strength")

# Create synthetic features
X["FCRatio"] = X["fineagg"] / X["coarseagg"]
X["AggCmtRatio"] = (X["coarseagg"] + X["fineagg"]) / X["cement"]
X["WtrCmtRatio"] = X["water"] / X["cement"]

# Train and score model on dataset with additional ratio features
model = RandomForestRegressor(criterion="absolute_error", random_state=0)
score = cross_val_score(
    model, X, y, cv=5, scoring="neg_mean_absolute_error"
)
score = -1 * score.mean()

print(f"MAE Score with Ratio Features: {score:.4}")

NameError: name 'df' is not defined

And sure enough, performance improved! This is evidence that these new ratio features exposed important information to the model that it wasn't detecting before.
#### Continue
We've seen that engineering new features can improve model performance. But how do you identify features in the dataset that might be useful to combine? __Discover useful features__ with mutual information.

## 2.- Mutual Information
- Locate features with the most potential.

### Intro
- First encountering a new dataset can feel overhelming

1. a great first step is construct a ranking with a __feature utility metric__ ( a function measuring associations between a feature and the target).
2. Then you can choose a smaller set of the most useful features to develop initially and have more confidence that your time will be well spent.
3. The metric we'll use is called "mutual information". 

#### Mutual information
- is a lot like correlation in that it measures a relationship between two quantities. The advantage of mutual information is that it can detect any kind of relationship, while correlation only detects linear relationships.
- Mutual information is a great general-purpose metric and especially useful at the start of feature development when you might not know what model you'd like to use yet. It is:
  1. easy to use an interpret.
  2. computationally efficient,
  3. thererically well-founded,
  4. resistant to overfitting, and,
  5. able to detect any kind of relationship.

### Mutual Information and What It Measures

In [34]:
urlA = 'https://github.com/jmonti-gh/Datasets/blob/main/test.csv'
urlB = 'https://github.com/jmonti-gh/Datasets/blob/227517a198b4b686f711fd84e95c81eb46a0a526/test.csv'
#https://github.com/jmonti-gh/Datasets/blob/227517a198b4b686f711fd84e95c81eb46a0a526/test.csv

In [38]:
import pandas as pd
import zipfile as zfm
import requests

# df = pd.read_csv(urlB)
# df

In [60]:
repo_owner = 'jmonti-gh'
repo_name = 'Datasets'
file_path = 'yourCsvFile.csv'

url = f'https://raw.githubusercontent.com/{repo_owner}/{repo_name}/main/{file_path}'
#https://github.com/jmonti-gh/Datasets/blob/main/test.csv
#https://github.com/juliencohensolal/BankMarketing/blob/master/rawData/bank-additional-full.csv

url1 = 'https://raw.githubusercontent.com/jmonti-gh/Datasets/main/test.csv'
url2 = 'https://raw.githubusercontent.com/jmonti-gh/Datasets/227517a198b4b686f711fd84e95c81eb46a0a526/test.csv'

url = 'https://raw.githubusercontent.com/juliencohensolal/BankMarketing/master/rawData/bank-additional-full.csv'

url3 = 'https://raw.githubusercontent.com/jmonti-gh/Datasets/main/bank-additional-full.csv'

data = pd.read_csv(url1, sep=",") # use sep="," for coma separation. 
data.describe()

EmptyDataError: No columns to parse from file

In [41]:
# url = 'http://winterolympicsmedals.com/medals.csv'
# r = requests.get(url)
# # text = r.iter_lines()
# # reader = csv.reader(text, delimiter=',')
# df = pd.read_csv(r)
# df

In [42]:
url='https://raw.githubusercontent.com/juliencohensolal/BankMarketing/master/rawData/bank-additional-full.csv'
data = pd.read_csv(url,sep=";") # use sep="," for coma separation. 
data.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,40.02406,258.285010,2.567593,962.475454,0.172963,0.081886,93.575664,-40.502600,3.621291,5167.035911
std,10.42125,259.279249,2.770014,186.910907,0.494901,1.570960,0.578840,4.628198,1.734447,72.251528
min,17.00000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.00000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.00000,180.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.00000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.00000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


In [25]:
import requests
import zipfile as zfm

url1 = 'https://github.com/jmonti-gh/Datasets/blob/main/Automobile_data.zip'
url2 = 'https://github.com/jmonti-gh/Datasets/blob/227517a198b4b686f711fd84e95c81eb46a0a526/Automobile_data.zip'
# zipfile = requests.get(url2)
# #zipfile = 'files/ConcreteCompressiveStrength.zip'

# with zfm.ZipFile(zipfile) as zf:
#     dir = zf.printdir()
#     lst = zf.namelist()
    
# print(dir)
# print(lst)

In [26]:
# import urllib.request

# url1 = 'https://github.com/jmonti-gh/Datasets/blob/main/Automobile_data.zip'
# url2 = 'https://github.com/jmonti-gh/Datasets/blob/227517a198b4b686f711fd84e95c81eb46a0a526/Automobile_data.zip'
# zipfile = urllib.request.urlopen(url2)

In [29]:
import urllib3

http = urllib3.PoolManager()
response = http.request('GET', url1)
zipfile = response.data.decode('utf-8')

In [30]:
with zfm.ZipFile(zipfile) as zf:
    dir = zf.printdir()
    lst = zf.namelist()
    
print(dir)
print(lst)

FileNotFoundError: [Errno 2] No such file or directory: '\n\n\n\n\n\n\n<!DOCTYPE html>\n<html lang="en" data-color-mode="auto" data-light-theme="light" data-dark-theme="dark" data-a11y-animated-images="system">\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch" href="https://github.githubassets.com">\n  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">\n  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">\n  <link rel="preconnect" href="https://github.githubassets.com" crossorigin>\n  <link rel="preconnect" href="https://avatars.githubusercontent.com">\n\n  \n\n  <link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/light-0946cdc16f15.css" /><link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/dark-3946c959759a.css" /><link data-color-theme="dark_dimmed"

In [ ]:
https://github.com/jmonti-gh/Datasets/blob/227517a198b4b686f711fd84e95c81eb46a0a526/Automobile_data.zip